<a href="https://colab.research.google.com/github/AliHasan-786/NextWordPredictionModel/blob/main/NextWordPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import necessary libraries
import tempfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

#Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
#Tokenize the text to create a sequence of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
#Create input-output pairs by splitting the text into sequence of tokens and forming n-grams from the sequences
input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
#Split the sequences into input and output
x = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
#Convert the output to one-hot encode vectors
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
#Build a neural network architecture to train the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation = 'softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 210s 69ms/step - loss: 6.2370 - accuracy: 0.0772
Epoch 2/100
3010/3010 [==============================] - 212s 70ms/step - loss: 5.5235 - accuracy: 0.1224
Epoch 3/100
3010/3010 [==============================] - 208s 69ms/step - loss: 5.1393 - accuracy: 0.1462
Epoch 4/100
3010/3010 [==============================] - 201s 67ms/step - loss: 4.8056 - accuracy: 0.1638
Epoch 5/100
3010/3010 [==============================] - 192s 64ms/step - loss: 4.5039 - accuracy: 0.1814
Epoch 6/100
3010/3010 [==============================] - 192s 64ms/step - loss: 4.2185 - accuracy: 0.2021
Epoch 7/100
3010/3010 [==============================] - 192s 64ms/step - loss: 3.9499 - accuracy: 0.2251
Epoch 8/100
3010/3010 [==============================] - 191s 64ms/step - loss: 3.6939 - accuracy: 0.2549
Epoch 9/100
3010/3010 [==============================] - 190s 63ms/step - loss: 3.4527 - accuracy: 0.2884
Epoch 10/100
3010/3010 [======================

In [ ]:
#Write your own seed text
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word


1/1 [==============================] - 0s 44ms/step
